##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ragged tensorsとは


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/ragged_tensor"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/ragged_tensor.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## セットアップ


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import math
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

## Overview

様々なデータは本来、バラバラな形を持っているはずで、またTensorについてもそうであるべきです。
*Ragged Tensor*とはTensorFlowにおけるネストされたリスト型のような可変長のデータを扱うことができます。
Ragged Tensorを使えば、可変長データを簡単に扱うことができます。
例えば、


*   映画の役者リストのような可変長のデータ
*   文章や映像といった、時系列データのまとまり
*   章、段落、文章、単語といったヒエラルキー構造な入力データ
*   Protocol Buffersのような個々の構造体データ

### Ragged Tensorでできること


Ragged tensorsは、`tf.add`や`tf.reduce_mean`のような数学的オペレーション、`tf.concat`や
`tf.tile`といった行列オペレーションを含む、100以上のTensorFlowオペレーションに対応しています。



In [0]:
digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
words = tf.ragged.constant([["So", "long"], ["thanks", "for", "all", "the", "fish"]])
print(tf.add(digits, 3))
print(tf.reduce_mean(digits, axis=1))
print(tf.concat([digits, [[5, 3]]], axis=0))
print(tf.tile(digits, [1, 2]))
print(tf.strings.substr(words, 0, 2))

Ragged Tensor特有の、Factory MethodsやConversion Methods、値を変換するオペレーションも存在
します。 リストオペレーションについては`tf.ragged`パッケージのドキュメントを参考にしてください。

通常のテンソルのように、Pythonライクなインデックス指定を行ったり、Slice指定を扱うこと
ができます。詳しくは**インデックス指定**の項を参考にしてください。


In [0]:
print(digits[0])       # 最初の行

In [0]:
print(digits[:, :2])   # 各行、最初の２つの値を取得

In [0]:
print(digits[:, -2:])  # 各行、最後の２つの値を取得

また、通常のTensorのように、Python算術演算子、比較演算子を使うことができます。詳細は、
オーバーロード演算子を参考にしてください。


In [0]:
print(digits + 3)

In [0]:
print(digits + tf.ragged.constant([[1, 2, 3, 4], [], [5, 6, 7], [8], []]))

もし、`RaggedTensor`の各要素に変換をかけたい場合は、`tf.ragged.map_flat_values`を使うことが
できます。これは`function`と`RaggedTensor`を引数にもち、下記のように変換をすることができます。


In [0]:
times_two_plus_one = lambda x: x * 2 + 1
print(tf.ragged.map_flat_values(times_two_plus_one, digits))

### Ragged Tensorを作る
Ragged Tensorを作る一番手っ取り早い方法は、`tf.ragged.constant`を使うことです。
Pythonのネストされた`list`からRagged Tensorを作成することができます。


In [0]:
sentences = tf.ragged.constant([
    ["Let's", "build", "some", "ragged", "tensors", "!"],
    ["We", "can", "use", "tf.ragged.constant", "."]])
print(sentences)

In [0]:
paragraphs = tf.ragged.constant([
    [['I', 'have', 'a', 'cat'], ['His', 'name', 'is', 'Mat']],
    [['Do', 'you', 'want', 'to', 'come', 'visit'], ["I'm", 'free', 'tomorrow']],
])
print(paragraphs)

Ragged Tensorは２つの要素から構成されています。それはデータを構成する*values*Tensorと、
それを行に分割していく*row-partitioning*Tensorです。 分割の仕方によって３つのClassメソッド、
`tf.RaggedTensor.from_value_rowids`、`tf.RaggedTensor.from_row_lengths`および、
`tf.RaggedTensor.from_row_splits`が定義されています。

#### `tf.RaggedTensor.from_value_rowids`
もし、各要素がどの列に入るのか事前に指定したい場合は、`value_rowids`による分割用Tensorを用いて
`RaggedTensor`をビルドすることができます。

![value_rowids](https://www.tensorflow.org/images/ragged_tensors/value_rowids.png)

In [0]:
print(tf.RaggedTensor.from_value_rowids(
    values=[3, 1, 4, 1, 5, 9, 2, 6],
    value_rowids=[0, 0, 0, 0, 2, 2, 2, 3]))

#### `tf.RaggedTensor.from_row_lengths`

長さを用いて列に分割したいのなら、`row_lengths`による分割用テンソルを用いて`RaggedTensor`をビルドできます。

![row_lengths](https://www.tensorflow.org/images/ragged_tensors/row_lengths.png)

In [0]:
print(tf.RaggedTensor.from_row_lengths(
    values=[3, 1, 4, 1, 5, 9, 2, 6],
    row_lengths=[4, 0, 3, 1]))

#### `tf.RaggedTensor.from_row_splits`

分割点を用いて列に分割したいのなら、`row_splits`による分割用テンソルを用いて`RaggedTensor`
をビルドできます。

![row_splits](https://www.tensorflow.org/images/ragged_tensors/row_splits.png)

In [0]:
print(tf.RaggedTensor.from_row_splits(
    values=[3, 1, 4, 1, 5, 9, 2, 6],
    row_splits=[0, 4, 4, 7, 8]))

また、その他のビルド方法に関しては、`tf.RaggedTensor`クラスのドキュメントを参考にしてください。 

### RaggedTensorに入れられるデータ

`RaggedTensor`は通常の`Tensor`のように、全てのデータは必ず同じ型でなければなりません。
また、ネストの深さも一様に同じでなければなりません。(ネストの深さとは、Tensorの*rank*のことです。)

In [0]:
print(tf.ragged.constant([["Hi"], ["How", "are", "you"]]))  # ok: type=string, rank=2

In [0]:
print(tf.ragged.constant([[[1, 2], [3]], [[4, 5]]]))        # ok: type=int32, rank=3

In [0]:
try:
  tf.ragged.constant([["one", "two"], [3, 4]])              # bad: 複数の type
except ValueError as exception:
  print(exception)

In [0]:
try:
  tf.ragged.constant(["A", ["B", "C"]])                     # bad: 複雑なネスト
except ValueError as exception:
  print(exception)

### ユースケース

今回の例では、可変長な全文章データを`RaggedTensor`を使ってunigramおよびbigramを構成し、  
さらにそれらをEmbedding表現を混ぜ込む方法を解説します。  
これらの機能についての詳しい解説は、`tf.ragged`パッケージのドキュメントを参考にしてください。

In [0]:
queries = tf.ragged.constant([['Who', 'is', 'Dan', 'Smith'],
                              ['Pause'],
                              ['Will', 'it', 'rain', 'later', 'today']])

# Embedding 用のテーブルを作成します。
num_buckets = 1024
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal([num_buckets, embedding_size],
                       stddev=1.0 / math.sqrt(embedding_size)))

# Embedding用に単語をそれぞれ数値(?)に変換し、Embeddingします。
word_buckets = tf.strings.to_hash_bucket_fast(queries, num_buckets)
word_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, word_buckets)                  # ①

# 文の最初と最後にbigramのためのマークをつけます。
marker = tf.fill([queries.nrows(), 1], '#')
padded = tf.concat([marker, queries, marker], axis=1)                       # ②

# 単語のbigramを構築し、bigram単語を数値(?)に変換の後、Embeddingします。
bigrams = tf.strings.join([padded[:, :-1],
                               padded[:, 1:]],
                              separator='+')                                # ③

bigram_buckets = tf.strings.to_hash_bucket_fast(bigrams, num_buckets)
bigram_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, bigram_buckets)                # ④

# 最後に文ごとにEmbeddingベクトルの平均値をとります。
all_embeddings = tf.concat([word_embeddings, bigram_embeddings], axis=1)    # ⑤
avg_embedding = tf.reduce_mean(all_embeddings, axis=1)                      # ⑥
print(avg_embedding)

![ragged_example](https://www.tensorflow.org/images/ragged_tensors/ragged_example.png)

## Ragged Tensorの詳細

### Raggedでありながら一部分固定長を持つ方法

*ragged tensor*というのも、詳細に言えば*raggedな次元*をもつテンソル表現の一つで、
ある特定の次元だけ可変長であるということもできます。　
例えば、`rt=[[3, 1, 4, 1], [], [5, 9, 2], [6], []]`の１つ内側の次元はRaggedな次元です。
それはもちろん、sliceをしてみる(`rt[0, :]`, ..., `rt[4, :]`)と違う長さのデータが出てくるから
です。ですが、内側のSliceでも次元が固定された状態の*uniform dimensions*をRaggedTensorで
構成することができます。
  
スライスによる次元は一定なので(?)一番外側の次元は必ず固定長です。(また、スライスによって次元
が変わる可能性がない)
また一番外側の次元と同じように、内側の次元でも固定長で扱うことができます。例えば、Word Embedding
では、Word部分となるRaggedTensorと、外側のBatchの次元と、さらにEmbeddingの次元も加えて
`[num_sentences, (num_words), embedding_size]`と表現できます。
ここでいう`(num_words)`がRaggedな次元となります。
  

![sent_word_embed](https://www.tensorflow.org/images/ragged_tensors/sent_word_embed.png)

また、RaggedTensorは複数のRaggedな次元を持つことができます。 例えば、文章を`[num_documents,
(num_paragraphs), (num_sentences), (num_words)]`のようにBatch化して、
処理することも可能です。(括弧で括られたところが、Raggedな次元と表現しています。)

#### Ragged tensor のShapeに関する制限

Ragged TensorのShapeは現時点では以下のように制限されています。

*   一番外側だけは固定長の次元（？）
*   一つ以上の可変長の次元が存在
*   0以上の固定長次元が存在

Note: これらの制限は実装の結果であり、今後制限が緩和されるかもしれません。

### Rank and ragged rank

Raggedな次元を含め、Ragged Tensorの全ての次元の数のことを***rank***と呼びます。また、Ragged
な次元だけを数えた次元数は***ragged rank***と呼びます。 graph execution mode(つまり、
non-earger mode)であるとき、Tensorのragged rankについては作成時に固定されます。つまり、
runtime valuesに依存することができない上、異なるsession runの中で動的に変えることもできません。
  
***potentially ragged tensor***とは、`tf.Tensor`か`tf.RaggedTensor`のどちらかの状態の
ことを指します。 また、`tf.Tensor`のragged rankは常に0と定義されています。

### RaggedTensor shapes

Ragged TensorのShapeについて述べる際、ragged な次元はカッコで括ります。例えば、上述の通り、
各単語のEmbedding表現のデータをもつRaggedTensorを
`[num_sentences, (num_words), embedding_size]`と表しています。
また、`RaggedTensor.shape`アトリビュートはRaggedな次元が`None`になった時に
`tf.TensorShape`を返します。


In [0]:
tf.ragged.constant([["Hi"], ["How", "are", "you"]]).shape

`tf.RaggedTensor.bounding_shape`は`RaggedTensor`がPadding等をして(意訳)固定長で入れる際
のshapeを返します。


In [0]:
print(tf.ragged.constant([["Hi"], ["How", "are", "you"]]).bounding_shape())

## Ragged vs sparse tensors

Ragged Tensorはsparse tensorとは*違いますが*、
不規則なテンソル型として密テンソルを表現する表現の一つであると考えて良いでしょう。

Ragged TensorとSparse Tensorの`concat`、`stack`そして`tile`の違いについて、
イラストによる例を以下に紹介しましょう。 Ragged Tensorでの結合では、
それぞれの行でそのままの形を保ったまま単一の行を形成していきます。


![ragged_concat](https://www.tensorflow.org/images/ragged_tensors/ragged_concat.png)


In [0]:
ragged_x = tf.ragged.constant([["John"], ["a", "big", "dog"], ["my", "cat"]])
ragged_y = tf.ragged.constant([["fell", "asleep"], ["barked"], ["is", "fuzzy"]])
print(tf.concat([ragged_x, ragged_y], axis=1))

しかし、sparse tensorでの結合は、以下のイラストで示す通り、密テンソルでの結合と同様に行われます。 
(Øは欠損値を表します)


![sparse_concat](https://www.tensorflow.org/images/ragged_tensors/sparse_concat.png)


In [0]:
sparse_x = ragged_x.to_sparse()
sparse_y = ragged_y.to_sparse()
sparse_result = tf.sparse.concat(sp_inputs=[sparse_x, sparse_y], axis=1)
print(tf.sparse.to_dense(sparse_result, ''))

またこのような区別がもたらす重要な違いについて、別の例として一つ紹介しましょう。
例えば、各行の平均値を知りたいとき、`tf.reduce_mean`オペレーションを使用しますが、
Ragged Tensorでは各行の長さで平均分母をとりますが、一方のSparse Tensorでは全体での長さの値が
平均分母をとります。(これは行の中で一番長い行か、別に設定されたそれ以上の値をとります）


## オーバーロード演算子
`Ragged Tensor`　クラスはPythonの算術演算子や比較演算子などをオーバーロードしています。
要素同士の計算については簡単に実装することができます。

In [0]:
x = tf.ragged.constant([[1, 2], [3], [4, 5, 6]])
y = tf.ragged.constant([[1, 1], [2], [3, 3, 3]])
print(x + y)

オーバーロード演算子は要素ごとの計算を行うため、入力は全て同じShapeを持っているか、
ブロードキャスト的に入力できる必要があります。 一番シンプルなブロードキャストの例が以下の通りで、
単一の数字を入力することでRagged Tensorの各要素に入力することができます。

In [0]:
x = tf.ragged.constant([[1, 2], [3], [4, 5, 6]])
print(x + 3)

もっと応用的なケースについては、**ブロードキャスト**の項を参考にしてください。

Ragged Tensorのオーバーロード演算子は、通常の`Tensor`と同じように以下の演算子をサポートしています。
単項演算子`-`、 `~`そして`abs()`　および　バイナリ演算子`+`, `-`, `*`, `/`,
`//`, `%`, `**`, `&`, `|`, `^`, `==`, `<`, `<=`, `>`,`>=`が対応しています。


## インデックス指定

Ragged TensorはPythonライクなインデックス指定や、多次元的なインデックス指定、
Sliceを指定することができます。2次元、3次元でのRagged Tensorの例を以下に示します。


### 2次元Ragged Tensor(1次元だけRaggedな次元)のインデックス指定

In [0]:
queries = tf.ragged.constant(
    [['Who', 'is', 'George', 'Washington'],
     ['What', 'is', 'the', 'weather', 'tomorrow'],
     ['Goodnight']])
print(queries[1])

In [0]:
print(queries[1, 2])                # 単語が出現

In [0]:
print(queries[1:])                  # 1行目以外の全て

In [0]:
print(queries[:, :3])               # 各行最初の3単語ずつ

In [0]:
print(queries[:, -2:])              # 各行最後に2単語ずつ

### 3次元Ragged Tensor(2次元がRaggedな次元)のインデックス指定

In [0]:
rt = tf.ragged.constant([[[1, 2, 3], [4]],
                         [[5], [], [6]],
                         [[7]],
                         [[8, 9], [10]]])

In [0]:
print(rt[1])                        # 第二行目 (2次元のRagged Tensor)

In [0]:
print(rt[3, 0])                     # 第四行目の最初の要素(1次元のRagged Tensor)

In [0]:
print(rt[:, 1:3])                   # 各行の1~3までの要素(3次元のRagged Tensor)

In [0]:
print(rt[:, -1:])                   # 各行最後の要素(3次元のRagged Tensor)


`Ragged Tensor`は多次元的なインデックス指定とSlice指定をサポートしています。しかし、制約もあります。
Raggedな次元でのインデックス指定だけはできません。これはインデックス指定時に、
値が存在しているかどうかが不明確だからです。この場合、`IndexError`を出すべきか、
与えられた初期値を出すべきか、はたまたスキップして少ない行のテンソルを渡すべきか(??)が、
不明瞭だからです。
[guiding principles of Python](https://www.python.org/dev/peps/pep-0020/)
によれば、曖昧さを否定せよと書かれているので(??)、
今の所現在では、Raggedな次元へのインデックス指定はできないように実装しています。


## テンソルへの変換

`Ragged Tensor`クラスは`tf.Tensor`や`tf.SparseTensors`といった、
テンソル型への変換をサポートしています。


In [0]:
ragged_sentences = tf.ragged.constant([
    ['Hi'], ['Welcome', 'to', 'the', 'fair'], ['Have', 'fun']])
print(ragged_sentences.to_tensor(default_value=''))

In [0]:
print(ragged_sentences.to_sparse())

In [0]:
x = [[1, 3, -1, -1], [2, -1, -1, -1], [4, 5, 8, 9]]
print(tf.RaggedTensor.from_tensor(x, padding=-1))

In [0]:
st = tf.SparseTensor(indices=[[0, 0], [2, 0], [2, 1]],
                     values=['a', 'b', 'c'],
                     dense_shape=[3, 3])
print(tf.RaggedTensor.from_sparse(st))

## Ragged Tensorの実数値化

### Eager execution

Eager executionモードでは、Ragged Tensorは即座に実数値化されます。
Ragged Tensorの実数値にアクセスする方法は以下の通りです。

*  `tf.RaggedTensor.to_list()`メソッドを使用する。これはPythonのリストに変換されます。

In [0]:
rt = tf.ragged.constant([[1, 2], [3, 4, 5], [6], [], [7]])
print(rt.to_list())

*   インデックス指定を行う。これはRaggedではない次元でインデックス指定を行うことにより、
    `EagerTensor`を返します。それの`numpy()`メソッドを使うことにより、
    値を直接取得することができます。
    

In [0]:
print(rt[1].numpy())

*   `tf.RaggedTensor.values`と`tf.RaggedTensor.row_splits`(または、
    `tf.RaggedTensor.row_lengths()`や`tf.RaggedTensor.value_rowids()`など)
    を使用することにより、
    Ragged Tensorを通常のテンソルに分解することも可能です。

In [0]:
print(rt.values)

In [0]:
print(rt.row_splits)

### ブロードキャスト

ブロードキャストとは、違うShapeを持つテンソルとのオペレーションにおいて、
要素ごとにオペレーションを実行できるように変換をかけることを指します。
ブロードキャストに関する詳細は以下のドキュメントを参考にしてください。

*   [Numpy: Broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)
*   `tf.broadcast_dynamic_shape`
*   `tf.broadcast_to`

互換性のある二つの入力、`x`と`y`を用いてブロードキャストの基本的な処理ステップを例として説明します。

1.  もし、`x`と`y`が違う次元数であった場合、同値になるまで外側の次元をShape1として追加していきます。

2.  `x`と`y`のそれぞれの次元数において、違うShapeを持っていた場合は

    *   `x`または`y`が次元`d`においてShape=1であった場合、同じ値を複製してもう一つの変数に
        オペレーションできるよう、入力Shapeを合わせます。

    *   そうでない場合は、`x`と`y`はブロードキャストできないとして例外を出力します
        

固定長な次元のテンソルのサイズ(??)は単一の数値として表現されますが、Ragged Tensorのサイズは
長さの値が入ったlistとして表現できます。(???)

#### ブロードキャストの例

In [0]:
# x       (2D ragged):  2 x (num_rows)
# y       (scalar)
# result  (2D ragged):  2 x (num_rows)
x = tf.ragged.constant([[1, 2], [3]])
y = 3
print(x + y)

In [0]:
# x         (2d ragged):  3 x (num_rows)
# y         (2d tensor):  3 x          1
# Result    (2d ragged):  3 x (num_rows)
x = tf.ragged.constant(
   [[10, 87, 12],
    [19, 53],
    [12, 32]])
y = [[1000], [2000], [3000]]
print(x + y)

In [0]:
# x      (3d ragged):  2 x (r1) x 2
# y      (2d ragged):         1 x 1
# Result (3d ragged):  2 x (r1) x 2
x = tf.ragged.constant(
    [[[1, 2], [3, 4], [5, 6]],
     [[7, 8]]],
    ragged_rank=1)
y = tf.constant([[10]])
print(x + y)

In [0]:
# x      (3d ragged):  2 x (r1) x (r2) x 1
# y      (1d tensor):                    3
# Result (3d ragged):  2 x (r1) x (r2) x 3
x = tf.ragged.constant(
    [
        [
            [[1], [2]],
            [],
            [[3]],
            [[4]],
        ],
        [
            [[5], [6]],
            [[7]]
        ]
    ],
    ragged_rank=2)
y = tf.constant([10, 20, 30])
print(x + y)

以下は、ブロードキャストが効かない例です。


In [0]:
# x      (2d ragged): 3 x (r1)
# y      (2d tensor): 3 x    4  # trailing dimensions do not match
x = tf.ragged.constant([[1, 2], [3, 4, 5, 6], [7]])
y = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

In [0]:
# x      (2d ragged): 3 x (r1)
# y      (2d ragged): 3 x (r2)  # ragged dimensions do not match.
x = tf.ragged.constant([[1, 2, 3], [4], [5, 6]])
y = tf.ragged.constant([[10, 20], [30, 40], [50]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

In [0]:
# x      (3d ragged): 3 x (r1) x 2
# y      (3d ragged): 3 x (r1) x 3  # trailing dimensions do not match
x = tf.ragged.constant([[[1, 2], [3, 4], [5, 6]],
                        [[7, 8], [9, 10]]])
y = tf.ragged.constant([[[1, 2, 0], [3, 4, 0], [5, 6, 0]],
                        [[7, 8, 0], [9, 10, 0]]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

## Ragged Tensorのエンコード

Ragged Tensorは`Ragged Tensor`クラスによってエンコードされます。
つまり`Ragged Tensor`は2つの構成要素で成り立っています。

*   均一なデータを持つ`values`テンソル
    
*   均一な直列データである`values`テンソルを行に分割する`row_splits`ベクトル  
    特に、Ragged Tensorにおける`rt[i]`とは、
    `rt.values[rt.row_splits[i]:rt.row_splits[i+1]]`と同値です。

![ragged_encoding](https://www.tensorflow.org/images/ragged_tensors/ragged_encoding.png)



In [0]:
rt = tf.RaggedTensor.from_row_splits(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_splits=[0, 4, 4, 6, 7])
print(rt)

### 複数のRaggedな次元のRagged Tensor

Ragged Tensorは複数のRaggedな次元を、`Ragged Tensor`の`values`テンソルを使って、
エンコードすることができます。 ネストされた`Ragged Tensor`は新たにもう一つRaggedな次元を追加します。

![ragged_rank_2](https://www.tensorflow.org/images/ragged_tensors/ragged_rank_2.png)

In [0]:
rt = tf.RaggedTensor.from_row_splits(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 3, 5, 9, 10]),
    row_splits=[0, 1, 1, 5])
print(rt)
print("Shape: {}".format(rt.shape))
print("Number of ragged dimensions: {}".format(rt.ragged_rank))

生成関数である`tf.RaggedTensor.from_nested_row_splits`は、複数のRaggedな次元をもつ、
Ragged Tensorを直接作成するための機能です。
`row_splits`テンソル（vectorでは?)を複数入力することで実現できます。

In [0]:
rt = tf.RaggedTensor.from_nested_row_splits(
    flat_values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    nested_row_splits=([0, 1, 1, 5], [0, 3, 3, 5, 9, 10]))
print(rt)

### 内側で固定長な次元を作成

Ragged Tensorは内側の次元に、`tf.Tensor`を`values`に用いることで、
均一で固定長な次元をエンコードすることができます。

![uniform_inner](https://www.tensorflow.org/images/ragged_tensors/uniform_inner.png)

In [0]:
rt = tf.RaggedTensor.from_row_splits(
    values=[[1, 3], [0, 0], [1, 3], [5, 3], [3, 3], [1, 2]],
    row_splits=[0, 3, 4, 6])
print(rt)
print("Shape: {}".format(rt.shape))
print("Number of ragged dimensions: {}".format(rt.ragged_rank))

### 他の行分割スキーマ

`Ragged Tensor`は、基本的に`row_splits`を用いることで均一なデータを行に分割して保存します。
しかし、`Ragged Tensor`は他の行分割スキーマもサポートしています。これらは、使用するデータによって、
様々に変えることができるので、さらに便利にエンコードすることができます。
`Ragged Tensor`は内部的に、それらのスキーマを効率化のために使用することがあります。

<dl>
  <dt>Row lengths</dt>
    <dd>`row_lengths`はベクトルのShapeが`[nrows]`で、各行の長さを指定するやり方です。</dd>

  <dt>Row starts</dt>
    <dd>`row_starts` はベクトルのShapeが `[nrows]`で、各行のスタートを記録したものです。
    これは`row_splits[:-1]`と同値です。</dd>

  <dt>Row limits</dt>
    <dd>`row_limits` はベクトルのShapeが `[nrows]`で, 各行の終わりを記録したものです。
    これは`row_splits[1:]`と同値です。</dd>

  <dt>Row indices and number of rows</dt>
    <dd>(独自訳)エンコードには2つの入力が必要で、`value_rowids`と`nrows`が必要です。
    `value_rowids` はベクトルのShapeが`[nvals]`で、要素ごとにどの行に入るかが、
    明記されたベクトルです。特に、全ての行`rt[row]`が全ての要素`rt.values[j]`を保持しているとき、
    つまり`value_rowids[j]==row`のとき、
    `nrows`は`Ragged Tensor`の行の中で特定された一つの整数値
    を持ちます。　また`nrows`が使用される時は、空の行を示すときに使用されます。

</dl>

以下に例を示します。以下のRagged Tensorは全て同値にエンコードされます。

In [0]:
values = [3, 1, 4, 1, 5, 9, 2, 6]
print(tf.RaggedTensor.from_row_splits(values, row_splits=[0, 4, 4, 7, 8, 8]))
print(tf.RaggedTensor.from_row_lengths(values, row_lengths=[4, 0, 3, 1, 0]))
print(tf.RaggedTensor.from_row_starts(values, row_starts=[0, 4, 4, 7, 8]))
print(tf.RaggedTensor.from_row_limits(values, row_limits=[4, 4, 7, 8, 8]))
print(tf.RaggedTensor.from_value_rowids(
    values, value_rowids=[0, 0, 0, 0, 2, 2, 2, 3], nrows=5))

Ragged Tensorクラスは、それぞれの行分割ベクトルを生成するメソッドを定義しています。

In [0]:
rt = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
print("      values: {}".format(rt.values))
print("  row_splits: {}".format(rt.row_splits))
print(" row_lengths: {}".format(rt.row_lengths()))
print("  row_starts: {}".format(rt.row_starts()))
print("  row_limits: {}".format(rt.row_limits()))
print("value_rowids: {}".format(rt.value_rowids()))

(注意：`tf.RaggedTensor.values` および`tf.RaggedTensors.row_splits`はクラスのプロパティ
として保持されますが、そのほかの行分割ベクトルを参照する際は、メソッドが呼び出されます。
これは`row_splits`は主としての行分割方法として表現できることを反映したもので
他の行分割ベクトルは呼び出されるごとに、生成しているということになります。)

それぞれの行分割スキーマの長所と短所をそれぞれ述べると、

+ **効果的なインデックス指定をしたい場合**:
    `row_splits`や`row_starts`および`row_limits`スキーマは、
    一定時間でインデックス指定が可能です。また、`value_rowids`や
    `row_lengths`ではそれができません。

+ **エンコードサイズを短くしたい場合**:
    `value_rowids`スキーマでは、空の行が多い場合に効果的にエンコードすることができます。
    また他の4つのエンコードスキーマでは、長い行がたくさんある場合に、単一の数字で表すことができるため、
    効率的にエンコードすることができます。

+ **結合を効率化したい場合**:
    `row_lengths`スキーマは結合したいときに効率的です。なぜなら、二つのテンソルを結合したときに、
    行の長さは変わらないからです。ちなみに、`row_splits`や`row_indices`は変わります(ほんと？？)

+ **互換性について**:
    `value_rowids`スキーマは`tf.segment_sum`のように
    [segmentation](../api_guides/python/math_ops.md#Segmentation)
    フォーマットが使われているオペレーションと互換性があります。(??)
    `row_limits`スキーマは`tf.sequence_mask`のようなオペレーションのフォーマットに
    互換性があります。(??)